In [2]:
import csv
import networkx as nx
import matplotlib.pyplot as plt
import matplotlib.colors as plcolor

In [3]:
def my_layout(G, scale=100):
    layout = {}
    for node in G.nodes():
        layout[node] = [G.nodes[node]['x'] / scale, G.nodes[node]['y'] / scale]
    return layout

In [4]:
cdict = {
    'red'   : [[0.0, 0.0, 0.0],
               [1.0, 0.0, 0.0]],
    'green' : [[0.0, 0.0, 0.0],
               [1.0, 0.0, 0.0]],
    'blue'  : [[0.0, 0.0, 0.0],
               [1.0, 0.0, 0.0]],
    'alpha' : [[0.0, 0.0, 0.0],
               [1.0, 1.0, 1.0]]
}
black_alpha = plcolor.LinearSegmentedColormap('black_alpha', cdict)

In [5]:
def total_degree(G, weight_str):
    sum_degree = 0
    for player, d in G.out_degree(weight=weight_str):
        sum_degree += d
    return sum_degree

In [18]:
def avg_distance(G, weight_str):
    short_ite = nx.all_pairs_dijkstra_path_length(G,weight=lambda u,v,d: 1 / d[weight_str])
    n = 0
    dist_tot = 0.0
    for s, d in short_ite:
        for k, v in d.items():
            dist_tot += v
            n += 1
    return dist_tot / n

In [19]:
def avg_position(G):
    sum_x = 0.0
    sum_y = 0.0
    num_node = G.number_of_nodes()
    for n in G.nodes():
        if 'x' in G.nodes[n].keys():
            sum_x += G.nodes[n]['x']
            sum_y += G.nodes[n]['y']
        else:
            num_node -= 1
    avg_pos = (sum_x / num_node, sum_y / num_node)
    return avg_pos

In [31]:
def important_pair(G, weight_str):
    nodes = list(G.nodes())
    sort_list = []
    for i in range(0, len(nodes)):
        for j in range(i+1, len(nodes)):
            if G.has_edge(nodes[i], nodes[j]) and G.has_edge(nodes[j], nodes[i]):
                sort_list.append( (G[nodes[i]][nodes[j]][weight_str]+G[nodes[j]][nodes[i]][weight_str], nodes[i], nodes[j]) )
    sort_list.sort(reverse = True)
    return sort_list[0:3]

In [32]:
def global_describe(G, weight_str):
    avg_clu = nx.average_clustering(G, weight=weight_str)
    tot_deg = total_degree(G, weight_str)
    max_cli = nx.graph_clique_number(G.to_undirected())
    avg_pos = avg_position(G)
    avg_dis = avg_distance(G, weight_str)
    imp_par = important_pair(G, weight_str)
    dict_des = {'avg_clustering': avg_clu, 'total_degree': tot_deg, 'clique_number': max_cli, 'avg_position_x': avg_pos[0], 'avg_position_y': avg_pos[1], 'avg_distance':avg_dis, 'important_pair': imp_par}
    return dict_des

In [21]:
def gen_cal_graph(team_name):
    G = nx.DiGraph()
    with open('2020_Problem_D_DATA/weight.csv') as f:
        f_csv = csv.DictReader(f)
        for row in f_csv:
            if team_name in row['OriginID'] and team_name in row['DestinationID']:
                if G.has_edge(row['OriginID'], row['DestinationID']):
                    G[row['OriginID']][row['DestinationID']]['weight'] += int(row['Weight'])
                else:
                    G.add_edge(row['OriginID'], row['DestinationID'], weight=int(row['Weight']))

    with open('2020_Problem_D_DATA/macro_location.csv') as f:
        f_csv = csv.DictReader(f)
        for row in f_csv:
            if team_name in row['OriginPlayerID']:
                G.nodes[row['OriginPlayerID']]['x'] = float(row['EventOrigin_x'])
                G.nodes[row['OriginPlayerID']]['y'] = float(row['EventOrigin_y'])
    total_dict = global_describe(G, 'weight')
    total_dict['teamID'] = team_name
    return total_dict

In [22]:
team_names = ['Huskies','Opponent1','Opponent2','Opponent3','Opponent4','Opponent5','Opponent6','Opponent7','Opponent8','Opponent9','Opponent10','Opponent11','Opponent12','Opponent13','Opponent14','Opponent15','Opponent16','Opponent17','Opponent18','Opponent19']
total_list = []
for name in team_names:
    total_list.append(gen_cal_graph(name))

In [24]:
headers = ['teamID', 'avg_distance', 'clique_number', 'avg_clustering', 'avg_position_y', 'avg_position_x', 'total_degree']
with open('global_indicator_dist.csv', 'w') as f:
    f_csv = csv.DictWriter(f, headers)
    f_csv.writeheader()
    f_csv.writerows(total_list)

In [33]:
def gen_cal_graph_match(match_num, team_name):
    G = nx.DiGraph()
    with open('2020_Problem_D_DATA/weight.csv') as f:
        f_csv = csv.DictReader(f)
        for row in f_csv:
            if int(row['MatchID']) is int(match_num) and team_name in row['OriginID'] and team_name in row['DestinationID']:
                if G.has_edge(row['OriginID'], row['DestinationID']):
                    G[row['OriginID']][row['DestinationID']]['weight'] += int(row['Weight'])
                else:
                    G.add_edge(row['OriginID'], row['DestinationID'], weight=int(row['Weight']))

    with open('2020_Problem_D_DATA/location.csv') as f:
        f_csv = csv.DictReader(f)
        for row in f_csv:
            if row['OriginPlayerID'] in G.nodes() and int(row['MatchID']) is int(match_num):
                G.nodes[row['OriginPlayerID']]['x'] = float(row['EventOrigin_x'])
                G.nodes[row['OriginPlayerID']]['y'] = float(row['EventOrigin_y'])
    
    total_dict = global_describe(G, 'weight')
    total_dict['teamID'] = team_name
    total_dict['matchID'] = match_num
    return total_dict

In [34]:
total_list = []
with open('2020_Problem_D_DATA/matches.csv') as f:
    f_csv = csv.DictReader(f)
    for row in f_csv:
        total_list.append(gen_cal_graph_match(row['MatchID'], 'Huskies'))
        total_list.append(gen_cal_graph_match(row['MatchID'], row['OpponentID']))

In [35]:
headers = ['matchID', 'teamID', 'avg_distance', 'important_pair', 'avg_clustering', 'total_degree', 'clique_number', 'avg_position_x', 'avg_position_y']
with open('global_indicator_dist.csv', 'w') as f:
    f_csv = csv.DictWriter(f, headers)
    f_csv.writeheader()
    f_csv.writerows(total_list)